#The Battle of the Neighborhoods - Part II 

##Insert Data and Install Packages

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [109]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(r.text, 'html.parser')
table=soup.find('table', attrs={'class':'wikitable sortable'})

headers=table.findAll('th')
for i, head in enumerate(headers): headers[i]=str(headers[i]).replace("<th>","").replace("</th>","").replace("\n","")

rows=table.findAll('tr')
rows=rows[1:len(rows)]

for i, row in enumerate(rows): rows[i] = str(rows[i]).replace("\n</td></tr>","").replace("<tr>\n<td>","")


df=pd.DataFrame(rows)
df[headers] = df[0].str.split("</td>\n<td>", n = 2, expand = True) 
df.drop(columns=[0],inplace=True)

df = df.drop(df[(df.Borough == "Not assigned")].index)

df.Neighbourhood.replace("Not assigned", df.Borough, inplace=True)


df.Neighbourhood.fillna(df.Borough, inplace=True)

df=df.drop_duplicates()

df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))

df.update(
    df.Borough.loc[
        lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))

df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('Toronto')
    ].str.replace(", Toronto",""))
df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('Toronto')
    ].str.replace("\(Toronto\)",""))

df_toronto = pd.DataFrame({'Postcode':df.Postcode.unique()})
df_toronto['Borough']=pd.DataFrame(list(set(df['Borough'].loc[df['Postcode'] == x['Postcode']])) for i, x in df_toronto.iterrows())
df_toronto['Neighborhood']=pd.Series(list(set(df['Neighbourhood'].loc[df['Postcode'] == x['Postcode']])) for i, x in df_toronto.iterrows())
df_toronto['Neighborhood']=df_toronto['Neighborhood'].apply(lambda x: ', '.join(x))
df_toronto.dtypes
df_toronto = df_toronto.rename(columns={'Postcode': 'Postal Code'})
df_toronto.head(20)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront , Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park (Toronto),Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,"Scarborough, Toronto","Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


##Print shape

In [19]:
df.shape

(211, 3)

##Print dataframe Toronto

In [111]:
df_toronto

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront , Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park (Toronto),Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,"Scarborough, Toronto","Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


####Combine neighborhoods with the same postal code.

In [22]:
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

df.head(20)

,Postcode,Borough,Neighbourhood
0,M1B,"Scarborough, Toronto","Rouge, Malvern"
1,M1C,"Scarborough, Toronto","Highland Creek , Rouge Hill, Port Union"
2,M1E,"Scarborough, Toronto","Guildwood, Morningside, West Hill"
3,M1G,"Scarborough, Toronto",Woburn
4,M1H,"Scarborough, Toronto",Cedarbrae
5,M1J,"Scarborough, Toronto",Scarborough Village
6,M1K,"Scarborough, Toronto","East Birchmount Park, Ionview, Kennedy Park"
7,M1L,"Scarborough, Toronto","Clairlea, Golden Mile, Oakridge"
8,M1M,"Scarborough, Toronto","Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,"Scarborough, Toronto","Birch Cliff, Cliffside West"


In [24]:
df.shape


(103, 3)

####Check to see if all values are relevant. 

In [25]:
print(df[df['Neighbourhood'] == 'Not assigned'])

Empty DataFrame
Columns: [Postcode, Borough, Neighbourhood]
Index: []


In [112]:
for index, row in df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']
        
df.tail(20)

,Postcode,Borough,Neighbourhood
251,M5X,Downtown Toronto,First Canadian Place
252,M5X,Downtown Toronto,Underground city
255,M8X,Etobicoke,The Kingsway
256,M8X,Etobicoke,Montgomery Road
257,M8X,Etobicoke,Old Mill North
262,M4Y,Downtown Toronto,Church and Wellesley
265,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
266,M8Y,Etobicoke,Humber Bay
267,M8Y,Etobicoke,King's Mill Park
268,M8Y,Etobicoke,Kingsway Park South East


In [27]:
df.shape

(103, 3)

In [29]:
df_code = pd.read_csv('http://cocl.us/Geospatial_data')

In [30]:
df_code

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [92]:
df_final = pd.merge(df,df_code,left_on='Postcode',right_on='Postal Code')
df_final.drop("Postal Code", axis=1, inplace=True)

##Print df_final

In [93]:
df_final

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,"Scarborough, Toronto","Rouge, Malvern",43.806686,-79.194353
1,M1C,"Scarborough, Toronto","Highland Creek , Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,"Scarborough, Toronto","Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,"Scarborough, Toronto",Woburn,43.770992,-79.216917
4,M1H,"Scarborough, Toronto",Cedarbrae,43.773136,-79.239476
5,M1J,"Scarborough, Toronto",Scarborough Village,43.744734,-79.239476
6,M1K,"Scarborough, Toronto","East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,"Scarborough, Toronto","Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,"Scarborough, Toronto","Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,"Scarborough, Toronto","Birch Cliff, Cliffside West",43.692657,-79.264848


In [37]:
df_final.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,"Scarborough, Toronto","Rouge, Malvern",43.806686,-79.194353
1,M1C,"Scarborough, Toronto","Highland Creek , Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,"Scarborough, Toronto","Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,"Scarborough, Toronto",Woburn,43.770992,-79.216917
4,M1H,"Scarborough, Toronto",Cedarbrae,43.773136,-79.239476


#Begin Data Analytics

In [39]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

##Import Python Libraries

In [42]:
from geopy.geocoders import Nominatim

from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

print('Libraries imported.')

Libraries imported.


##Use Geolocator

In [67]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude

print('Toronto is at geographical coordinates of latitude and longitude {}, {}.'.format(latitude_toronto, longitude_toronto))

Toronto is at geographical coordinates of latitude and longitude 43.653963, -79.387207.


In [55]:
CLIENT_ID = 'VB5SQ2GMN4T3YG3ZTSMF2GJHWR2YX0ZE2RLEW1CQQJ31ZRZH' 
CLIENT_SECRET = 'C4BNQGDEZS5V5FLSMLS1M25WMOUCNF1IWA3CWJF53O3VYTRI' 
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VB5SQ2GMN4T3YG3ZTSMF2GJHWR2YX0ZE2RLEW1CQQJ31ZRZH
CLIENT_SECRET:C4BNQGDEZS5V5FLSMLS1M25WMOUCNF1IWA3CWJF53O3VYTRI


##Find all venues

In [58]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

##List the boroughs in Toronto

In [68]:
venues_in_toronto = getNearbyVenues(names=df_final['Borough'],
                                   latitudes=df_final['Latitude'],
                                   longitudes=df_final['Longitude']
                                  )

Scarborough, Toronto
Scarborough, Toronto
Scarborough, Toronto
Scarborough, Toronto
Scarborough, Toronto
Scarborough, Toronto
Scarborough, Toronto
Scarborough, Toronto
Scarborough, Toronto
Scarborough, Toronto
Scarborough, Toronto
Scarborough, Toronto
Scarborough, Toronto
Scarborough, Toronto
Scarborough, Toronto
Scarborough, Toronto
Scarborough, Toronto
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
East York
East York
East Toronto
East York
East York
East York
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
Central Toronto
Central Toronto


##Give shape to the venues in Toronto

In [69]:
print(venues_in_toronto.shape)
toronto_venues.head()

(2242, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Scarborough, Toronto",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Scarborough, Toronto",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Scarborough, Toronto",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"Scarborough, Toronto",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Scarborough, Toronto",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


##Print the venues in Toronto (the first twenty rows)

In [70]:
venues_in_toronto.head(20)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Scarborough, Toronto",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Scarborough, Toronto",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Scarborough, Toronto",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"Scarborough, Toronto",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Scarborough, Toronto",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
5,"Scarborough, Toronto",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
6,"Scarborough, Toronto",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
7,"Scarborough, Toronto",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center
8,"Scarborough, Toronto",43.763573,-79.188711,Lawrence Ave E & Kingston Rd,43.767704,-79.189490,Intersection
9,"Scarborough, Toronto",43.763573,-79.188711,Eggsmart,43.767800,-79.190466,Breakfast Spot


In [71]:
venues_in_toronto.head(200)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Scarborough, Toronto",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Scarborough, Toronto",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Scarborough, Toronto",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"Scarborough, Toronto",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Scarborough, Toronto",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
5,"Scarborough, Toronto",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
6,"Scarborough, Toronto",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
7,"Scarborough, Toronto",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center
8,"Scarborough, Toronto",43.763573,-79.188711,Lawrence Ave E & Kingston Rd,43.767704,-79.189490,Intersection
9,"Scarborough, Toronto",43.763573,-79.188711,Eggsmart,43.767800,-79.190466,Breakfast Spot


In [72]:
venues_in_toronto

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Scarborough, Toronto",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Scarborough, Toronto",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Scarborough, Toronto",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"Scarborough, Toronto",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Scarborough, Toronto",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
5,"Scarborough, Toronto",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
6,"Scarborough, Toronto",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
7,"Scarborough, Toronto",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center
8,"Scarborough, Toronto",43.763573,-79.188711,Lawrence Ave E & Kingston Rd,43.767704,-79.189490,Intersection
9,"Scarborough, Toronto",43.763573,-79.188711,Eggsmart,43.767800,-79.190466,Breakfast Spot


####Make a count of the Toronto venues

In [105]:
venues_in_toronto.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Toronto,115,115,115,115,115,115
Downtown Toronto,1285,1285,1285,1285,1285,1285
East Toronto,122,122,122,122,122,122
East York,76,76,76,76,76,76
Etobicoke,73,73,73,73,73,73
Mississauga,11,11,11,11,11,11
North York,237,237,237,237,237,237
Queen's Park (Toronto),41,41,41,41,41,41
"Scarborough, Toronto",87,87,87,87,87,87


##Perform One Hot Encoding on the Toronto Neighborhoods

In [73]:
onehotencode_toronto = pd.get_dummies(venues_in_toronto[['Venue Category']], prefix="", prefix_sep="")

onehotencode_toronto['Neighborhood'] = venues_in_toronto['Neighborhood'] 

fixed_columns = [onehotencode_toronto.columns[-1]] + list(onehotencode_toronto.columns[:-1])
onehotencode_toronto = onehotencode_toronto[fixed_columns]

In [74]:
onehotencode_toronto.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
##Construct clusters

In [75]:
clustergrouped_toronto = onehotencode_toronto.groupby('Neighborhood').mean().reset_index()
clustergrouped_toronto

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Central Toronto,0.008696,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.008696,0.000000,0.000000,0.008696,0.000000,0.000000,0.000000,0.000000,0.000000
1,Downtown Toronto,0.002335,0.000000,0.000778,0.000778,0.000778,0.000778,0.001556,0.001556,0.001556,...,0.000000,0.011673,0.002335,0.000778,0.005447,0.000000,0.006226,0.000000,0.000778,0.000778
2,East Toronto,0.016393,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,East York,0.013158,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.013158,0.000000,0.013158,0.000000,0.000000,0.000000,0.000000
4,Etobicoke,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013699,0.000000
5,Mississauga,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,North York,0.000000,0.004219,0.000000,0.004219,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.004219,0.004219,0.008439,0.000000,0.000000,0.000000,0.004219,0.012658
7,Queen's Park (Toronto),0.024390,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024390,0.000000
8,"Scarborough, Toronto",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.022989,0.000000,0.000000,0.000000,0.000000,0.000000
9,West Toronto,0.005618,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.011236,0.000000,0.000000,0.011236,0.000000,0.005618,0.005618,0.000000,0.000000


In [80]:
num_top_venues = 5

for hood in clustergrouped_toronto['Neighborhood']:
    print("----"+hood+"----")
    temp = clustergrouped_toronto[clustergrouped_toronto['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.07
1  Sandwich Place  0.06
2            Park  0.05
3            Café  0.05
4             Gym  0.05


----Downtown Toronto----
                venue  freq
0         Coffee Shop  0.10
1                Café  0.06
2  Italian Restaurant  0.03
3               Hotel  0.03
4              Bakery  0.03


----East Toronto----
                venue  freq
0    Greek Restaurant  0.07
1         Coffee Shop  0.07
2  Italian Restaurant  0.05
3      Ice Cream Shop  0.04
4                Café  0.04


----East York----
                 venue  freq
0          Coffee Shop  0.07
1                 Park  0.05
2         Burger Joint  0.05
3  Sporting Goods Shop  0.04
4             Pharmacy  0.04


----Etobicoke----
                  venue  freq
0           Pizza Place  0.12
1        Sandwich Place  0.07
2              Pharmacy  0.05
3  Fast Food Restaurant  0.04
4                   Gym  0.04


----Mississauga----
                       venue  

In [81]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]



In [113]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Neighborhood'] = clustergrouped_toronto['Neighborhood']

for ind in np.arange(clustergrouped_toronto.shape[0]):
   venues_sorted.iloc[ind, 1:] = return_most_common_venues(clustergrouped_toronto.iloc[ind, :], num_top_venues)

venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Park,Gym,Café,Pizza Place,Dessert Shop,Sushi Restaurant,Clothing Store,Restaurant
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Bakery,Japanese Restaurant,Bar,Seafood Restaurant,Park
2,East Toronto,Coffee Shop,Greek Restaurant,Italian Restaurant,Café,Ice Cream Shop,Brewery,Pub,Pizza Place,Sandwich Place,Bakery
3,East York,Coffee Shop,Burger Joint,Park,Bank,Pizza Place,Pharmacy,Sporting Goods Shop,Fast Food Restaurant,Supermarket,Furniture / Home Store
4,Etobicoke,Pizza Place,Sandwich Place,Pharmacy,Park,Coffee Shop,Gym,Grocery Store,Café,Discount Store,Fast Food Restaurant
5,Mississauga,Hotel,Coffee Shop,Gym / Fitness Center,Mediterranean Restaurant,Middle Eastern Restaurant,Sandwich Place,Burrito Place,American Restaurant,Fried Chicken Joint,Donut Shop
6,North York,Coffee Shop,Clothing Store,Fast Food Restaurant,Park,Japanese Restaurant,Sandwich Place,Restaurant,Grocery Store,Pizza Place,Café
7,Queen's Park (Toronto),Coffee Shop,Park,Gym,Diner,College Auditorium,Seafood Restaurant,Sandwich Place,Burger Joint,Burrito Place,Restaurant
8,"Scarborough, Toronto",Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Indian Restaurant,Bakery,Middle Eastern Restaurant,Grocery Store,Intersection
9,West Toronto,Bar,Coffee Shop,Café,Italian Restaurant,Restaurant,Pizza Place,Bakery,Breakfast Spot,French Restaurant,Bookstore


In [ ]:
##Use k-means Clustering Algorithm 

In [120]:
#Specify the number of clusters
kclusters = 10

clustergrouped_toronto_clustering = clustergrouped_toronto.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(clustergrouped_toronto_clustering)

kmeans.labels_[0:10]

array([0, 4, 9, 8, 6, 3, 4, 5, 7, 4], dtype=int32)

In [119]:
venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ValueError: cannot insert Cluster Labels, already exists

##Label the Clusters

In [121]:
merged_toronto = df_final

merged_toronto = merged_toronto.join(venues_sorted.set_index('Neighborhood'), on='Borough')

merged_toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,"Scarborough, Toronto","Rouge, Malvern",43.806686,-79.194353,7,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Indian Restaurant,Bakery,Middle Eastern Restaurant,Grocery Store,Intersection
1,M1C,"Scarborough, Toronto","Highland Creek , Rouge Hill, Port Union",43.784535,-79.160497,7,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Indian Restaurant,Bakery,Middle Eastern Restaurant,Grocery Store,Intersection
2,M1E,"Scarborough, Toronto","Guildwood, Morningside, West Hill",43.763573,-79.188711,7,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Indian Restaurant,Bakery,Middle Eastern Restaurant,Grocery Store,Intersection
3,M1G,"Scarborough, Toronto",Woburn,43.770992,-79.216917,7,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Indian Restaurant,Bakery,Middle Eastern Restaurant,Grocery Store,Intersection
4,M1H,"Scarborough, Toronto",Cedarbrae,43.773136,-79.239476,7,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Indian Restaurant,Bakery,Middle Eastern Restaurant,Grocery Store,Intersection


In [122]:
merged_toronto.head(20)

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,"Scarborough, Toronto","Rouge, Malvern",43.806686,-79.194353,7,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Indian Restaurant,Bakery,Middle Eastern Restaurant,Grocery Store,Intersection
1,M1C,"Scarborough, Toronto","Highland Creek , Rouge Hill, Port Union",43.784535,-79.160497,7,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Indian Restaurant,Bakery,Middle Eastern Restaurant,Grocery Store,Intersection
2,M1E,"Scarborough, Toronto","Guildwood, Morningside, West Hill",43.763573,-79.188711,7,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Indian Restaurant,Bakery,Middle Eastern Restaurant,Grocery Store,Intersection
3,M1G,"Scarborough, Toronto",Woburn,43.770992,-79.216917,7,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Indian Restaurant,Bakery,Middle Eastern Restaurant,Grocery Store,Intersection
4,M1H,"Scarborough, Toronto",Cedarbrae,43.773136,-79.239476,7,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Indian Restaurant,Bakery,Middle Eastern Restaurant,Grocery Store,Intersection
5,M1J,"Scarborough, Toronto",Scarborough Village,43.744734,-79.239476,7,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Indian Restaurant,Bakery,Middle Eastern Restaurant,Grocery Store,Intersection
6,M1K,"Scarborough, Toronto","East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,7,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Indian Restaurant,Bakery,Middle Eastern Restaurant,Grocery Store,Intersection
7,M1L,"Scarborough, Toronto","Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,7,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Indian Restaurant,Bakery,Middle Eastern Restaurant,Grocery Store,Intersection
8,M1M,"Scarborough, Toronto","Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476,7,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Indian Restaurant,Bakery,Middle Eastern Restaurant,Grocery Store,Intersection
9,M1N,"Scarborough, Toronto","Birch Cliff, Cliffside West",43.692657,-79.264848,7,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Indian Restaurant,Bakery,Middle Eastern Restaurant,Grocery Store,Intersection


##Print the entire table of merged Toronto information

In [123]:
merged_toronto.head(103)

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,"Scarborough, Toronto","Rouge, Malvern",43.806686,-79.194353,7,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Indian Restaurant,Bakery,Middle Eastern Restaurant,Grocery Store,Intersection
1,M1C,"Scarborough, Toronto","Highland Creek , Rouge Hill, Port Union",43.784535,-79.160497,7,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Indian Restaurant,Bakery,Middle Eastern Restaurant,Grocery Store,Intersection
2,M1E,"Scarborough, Toronto","Guildwood, Morningside, West Hill",43.763573,-79.188711,7,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Indian Restaurant,Bakery,Middle Eastern Restaurant,Grocery Store,Intersection
3,M1G,"Scarborough, Toronto",Woburn,43.770992,-79.216917,7,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Indian Restaurant,Bakery,Middle Eastern Restaurant,Grocery Store,Intersection
4,M1H,"Scarborough, Toronto",Cedarbrae,43.773136,-79.239476,7,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Indian Restaurant,Bakery,Middle Eastern Restaurant,Grocery Store,Intersection
5,M1J,"Scarborough, Toronto",Scarborough Village,43.744734,-79.239476,7,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Indian Restaurant,Bakery,Middle Eastern Restaurant,Grocery Store,Intersection
6,M1K,"Scarborough, Toronto","East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,7,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Indian Restaurant,Bakery,Middle Eastern Restaurant,Grocery Store,Intersection
7,M1L,"Scarborough, Toronto","Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,7,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Indian Restaurant,Bakery,Middle Eastern Restaurant,Grocery Store,Intersection
8,M1M,"Scarborough, Toronto","Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476,7,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Indian Restaurant,Bakery,Middle Eastern Restaurant,Grocery Store,Intersection
9,M1N,"Scarborough, Toronto","Birch Cliff, Cliffside West",43.692657,-79.264848,7,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Chinese Restaurant,Pizza Place,Indian Restaurant,Bakery,Middle Eastern Restaurant,Grocery Store,Intersection
